# Fake-emo React

In fake emo react dataset we cropped the english samples and made the total dataset size as: - 7012 - real (3793) + fake (3219)

In [2]:
pwd

'/home/karish19471/btp/amr'

In [8]:
import json

In [10]:
train = json.load(open("fake-emo_amr/train.json", "rb"))

In [66]:
def get_s2c(data, sample2comments = {}):
    for sample in data:
        if sample['idx'] in sample2comments:
            sample2comments[sample['idx']]['replies'].append(sample['reply'])
        else:
            sample2comments[sample['idx']] = {}
            sample2comments[sample['idx']]['replies'] = [sample['reply']]
            sample2comments[sample['idx']]['label'] = sample['label']
            sample2comments[sample['idx']]['text'] = sample['text']
    return sample2comments

sample2comments = get_s2c(train)

In [67]:
import pandas as pd
df = pd.DataFrame(sample2comments).T

In [68]:
import numpy as np

In [69]:
df = df.set_index(np.arange(len(df)))

In [70]:
real = df[df['label']=='real']
fake = df[df['label']=='fake']

In [71]:
real['len'] = None

/tmp/ipykernel_3539999/3869150830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['len'] = None


In [76]:
for i, j in real.iterrows():
    if j['replies'][0]!='':
        real.at[i, 'len'] = len(j['replies'])
    else:
        real.at[i, 'len'] = 0

In [84]:
crop_real = real[real['len'] == 1].sample(frac = 0.3)

In [86]:
merged  = pd.concat([fake, crop_real])

In [87]:
del merged['len']

In [91]:
merged = merged.sample(frac = 1)

In [93]:
merged = merged.set_index(np.arange(len(merged)))

In [97]:
merged['label'].value_counts()

real    3793
fake    3219
Name: label, dtype: int64

In [101]:
merged['id'] = "fake_emo_"+merged.index.astype(str)

In [104]:
merged = merged[['id', 'text', 'replies', 'label']]

In [107]:
merged.to_csv("fake-emo_amr/fake-emo.csv", index = False)

# Antivax

final dataset size is (0, 2865) (1, 932)

In [9]:
cd btp/amr

/home/karish19471/btp/amr


In [10]:
import json
import preprocessor as p
import pandas as pd
import numpy as np
import glob

In [18]:
antivax = glob.glob("antivax_amr/comments/*.json")
len_comments = []
df = pd.DataFrame()
id_list, comments_list = [], []
for lv, news in enumerate(antivax):
    all_comments= []
    user_comment = json.load(open(news, 'rb'))
    for i in user_comment['data']:
        all_comments.append(p.clean(i['text']))

    if len(all_comments)>0:#considering only news articles with atleast one comment
        processed = "::".join(all_comments)
        comments_list.append(processed)
        id_list.append(news[:-1][news[:-1].rfind("/")+1:].split(".")[0])
    print(lv, end = '\r', flush = True)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [19]:
df['id'] =  id_list
df['comments'] = comments_list

In [24]:
df

,id,comments
0,1346889653669007361,Read today that they aren't doing the vaccine ...
1,1344313403242852353,Unproven vaccine? Wtf? SOMETHING!!!
2,1390453791363903489,At this point it may do just that::Man my arm ...
3,1387777120139718667,I would do it again in a heartbeat!::It was ro...
4,1390040566394540032,G Microchipped Bois Life
...,...,...
4289,1342225714070888449,Wdym
4290,1360385133421993987,I'd recommend tapping into some of those sweet...
4291,1388666725114122247,Read it again. The study intervention is the t...
4292,1372998962328657922,"So, the Russians are the Klingons.::So maybe t..."


In [43]:
text = pd.read_csv('antivax_amr/antivax.csv')
label = pd.read_csv("antivax_amr/Labeled/VaxMisinfoData.csv")

In [47]:
idx2text  = dict(zip(list(text['idx']), list(text['text'])))
idx2label  = dict(zip(list(label['id']), list(label['is_misinfo'])))

In [55]:
text, labels = [], []
for i, j in df.iterrows():
    text.append(idx2text.get(int(j['id']), None))
    labels.append(idx2label.get(int(j['id']), None))

In [58]:
df['text'] = text
df['labels'] = labels

In [60]:
df = df.dropna()

In [62]:
df['labels'].value_counts()

0    2865
1     932
Name: labels, dtype: int64

In [65]:
df = df.rename(columns = {'id':'tweet_id'})

In [67]:
df['id'] = "antivax_"+df.index.astype(str)

In [69]:
df = df[['id', 'tweet_id', 'text', 'comments', 'labels']]

In [72]:
df.to_csv("antivax_amr/antivax.csv", index = False)

In [73]:
df

,id,tweet_id,text,comments,labels
1,antivax_1,1344313403242852353,"They are doing this, so you beg for an unprove...",Unproven vaccine? Wtf? SOMETHING!!!,1
2,antivax_2,1390453791363903489,"Fully vaccinated, lets see if the 2nd dose “va...",At this point it may do just that::Man my arm ...,0
3,antivax_3,1387777120139718667,Big BIG feels #vaccinated #thankyouscience #Mo...,I would do it again in a heartbeat!::It was ro...,0
4,antivax_4,1390040566394540032,.@ZaddiHen grateful for the vaccine link. Neve...,G Microchipped Bois Life,0
5,antivax_5,1359290042821341184,just got an email about getting my covid vacci...,lol JK i have to wait months to get it now! yo...,0
...,...,...,...,...,...
4289,antivax_4289,1342225714070888449,If you drink redbull don’t worry about what’s ...,Wdym,0
4290,antivax_4290,1360385133421993987,Just got my first vaccine dose. One step close...,I'd recommend tapping into some of those sweet...,0
4291,antivax_4291,1388666725114122247,See page 67 - 69 on possible ‘shedding’ // vac...,Read it again. The study intervention is the t...,1
4292,antivax_4292,1372998962328657922,If you threaten to poison vaccines because som...,"So, the Russians are the Klingons.::So maybe t...",1


# figlang_reddit

NOT_SARCASM    2200
SARCASM        2200

In [11]:
import json

In [24]:
import pandas as pd    
reddit = pd.read_json(path_or_buf="figlang_reddit_amr/sarcasm_detection_shared_task_reddit_training.jsonl", lines=True)

In [25]:
reddit = reddit.rename(columns={'response':'text', 'context':'comments'})

In [93]:
lens = np.array([len(i) for i in list(reddit['comments'])])

In [100]:
reddit['id'] = "reddit_"+reddit.index.astype(str)

In [102]:
reddit

,label,text,comments,id
0,SARCASM,"Yeah I mean there's only one gender anyways, w...",[LPT: If you're worried about hurting someone'...,reddit_0
1,SARCASM,"Sounds like you don't like science, you theist...",[Promotional images for some guy's Facebook pa...,reddit_1
2,SARCASM,"Ofc play them in try mode, Blizzard were so ge...",[My friends won't play Dota2; I won't play LoL...,reddit_2
3,SARCASM,"I don't understand, Reddit told me that Hillar...",[Poll: Convention boosts Clinton to 11-point l...,reddit_3
4,SARCASM,"yeh, they're the reigning triple premiers, why...",[Wayne Ludbey: Jordan Lewis has the ultimate c...,reddit_4
...,...,...,...,...
4395,NOT_SARCASM,well you could've been adulting if you hadn't ...,[8-9ft man found in ancient indian burial moun...,reddit_4395
4396,NOT_SARCASM,Also they'll have to join the euro,[Second Scottish independence referendum 'on t...,reddit_4396
4397,NOT_SARCASM,plot: AI assists a cyborg in freelance investi...,"[Pinoy Cyborg by James Simmons, Mag-ingat sa r...",reddit_4397
4398,NOT_SARCASM,Some airlines proposed this but too much publi...,"[The logic here is flawless!, No it isn't, for...",reddit_4398


In [103]:
reddit = reddit[['id', 'text', 'comments', 'label']].sample(frac = 1)

In [105]:
reddit['label'].value_counts()

NOT_SARCASM    2200
SARCASM        2200
Name: label, dtype: int64

In [106]:
reddit.to_csv("figlang_reddit/reddit.csv", index = False)

# figlang_twitter

In [35]:
import preprocessor as pro

In [36]:
import pandas as pd    
twitter = pd.read_json(path_or_buf="figlang_twitter_amr/sarcasm_detection_shared_task_twitter_training.jsonl", lines=True)

In [37]:
twitter = twitter.rename(columns={'response':'text', 'context':'comments'})
twitter['id'] = "twitter_"+twitter.index.astype(str)
twitter = twitter[['id', 'text', 'comments', 'label']].sample(frac = 1)

In [38]:
for i, j in twitter.iterrows():
    twitter.at[i, 'text'] = pro.clean(j['text'])
    comm = j['comments']
    cleaned = []
    for o in comm:
        cleaned.append(pro.clean(o).strip())
    twitter.at[i, 'comments'] = "::".join(cleaned)


In [40]:
twitter.to_csv("figlang_twitter_amr/figlang_twitter.csv", index = False)

# hasoc

NONE    414
HOF     298

In [1]:
import glob

In [5]:
dirs = glob.glob("/home/karish19471/btp/amr/hasoc_amr/Data/task2_data/train/*/")

In [7]:
import preprocessor as p

In [8]:
data_comments = []
data_text = []
data_id = []
id2label = {}

In [11]:
for directory in dirs:
    tweets = glob.glob(f'{directory}*/')
    for tweet in tweets:
        data = json.load(open(f'{tweet}data.json', 'rb'))
        LABEL = json.load(open(f'{tweet}labels.json', 'rb'))
        for key in LABEL:
            id2label[key] = LABEL[key]
        # data = json.load(open(f'/home/karish19471/btp/amr/hasoc/Data/task2_data/train/bantwitter/1397101600460529665/data.json', 'rb'))
        text = p.clean(data['tweet'])
        tweet_id = data['tweet_id']
        extra_tweets_text = []
        extra_tweets_replies = []
        extra_tweets_ids = []
        comments = []
        for comment in data['comments']:
            tmp = comment.get('replies', -1)
            if tmp==-1:
                comments.append(p.clean(comment['tweet']))
            else:
                comments.append(p.clean(comment['tweet']))
                
                replies_list = []
                for replies in tmp:
                    comments.append(p.clean(replies['tweet']))
                    replies_list.append(p.clean(replies['tweet']))
                
                extra_tweets_text.append(p.clean(comment['tweet']))
                extra_tweets_ids.append(comment['tweet_id'])
                extra_tweets_replies.append("::".join(replies_list))

        extra_tweets_replies.append("::".join(comments))
        extra_tweets_ids.append(tweet_id)
        extra_tweets_text.append(text)

        data_comments.extend(extra_tweets_replies)
        data_text.extend(extra_tweets_text)
        data_id.extend(extra_tweets_ids)

In [219]:
len(data_id), len(data_text), len(data_comments)

(712, 712, 712)

In [220]:
hasoc = pd.DataFrame()
hasoc['twitter_id'] = data_id
hasoc['text'] = data_text
hasoc['comments'] = data_comments

In [221]:
hasoc['label'] = None
for i, j in hasoc.iterrows():
    j['label'] = id2label[str(j['twitter_id'])]

In [222]:
hasoc['label'].value_counts()

NONE    414
HOF     298
Name: label, dtype: int64

In [223]:
hasoc

,twitter_id,text,comments,label
0,1395727658323898368,Point,No*,NONE
1,1395695944583180293,Yes terrorism from Pakistan as well as RSS. I ...,"Terrorism from lefrist, missionaries and jihad...",NONE
2,1395703843162267648,"Accha! Climate Change to gaya, khatam!!",?,NONE
3,1395714606278975492,,?,NONE
4,1395742992019820552,Ab panditon ki dukaan bnd or paakhnd bhi bnd,Great I am waiting for the day.::Isaaio se jya...,HOF
...,...,...,...,...
707,1397949847609614342,India was able to screw Huawei when it came to...,Absolutely.. no nd thought on this issue::Plea...,NONE
708,1397979516585213953,So as a matter of protest please quit twitter.,Its a same logic pakistan niklo ..Why u want t...,NONE
709,1397999669607686145,"Sir, Twitter is the new East India Company, bu...",Simple...You can deactivate your acc in one cl...,NONE
710,1397914911431217152,Twitter is the new East India Companypretendin...,Twitter has been acting like a dictator and ba...,NONE


In [224]:
hasoc['id'] = "hasoc_"+hasoc.index.astype(str)
hasoc = hasoc[['id','twitter_id', 'text', 'comments', 'label']].sample(frac = 1)

In [227]:
hasoc.to_csv("hasoc/hasoc.csv", index = False)

# jing_gab

NON_HATE    7665
HATE        7665

In [298]:
import pandas as pd

In [307]:
gab = pd.read_csv("jing_gab_amr/gab.csv")

In [308]:
gab = gab.dropna()

In [337]:
import preprocessor as pro

In [338]:
hate_texts, hate_context, non_hate_texts, non_hate_context = [], [], [], []
for i, j in gab.iterrows():
    j['text'] = j['text'].replace('\t', '')
    texts = j['text'].strip().split('\n')
    new_texts = []
    for p in texts:
        new_texts.append(pro.clean(".".join(p.split(".")[1:]).strip()))
    texts = new_texts
    idx = eval(j['hate_speech_idx']) 

    h_texts  = [v for i, v in enumerate(texts) if i+1 in idx]
    n_h_texts = [v for i, v in enumerate(texts) if i+1 not in idx]

    try:
        source_n_h_text = n_h_texts[0]
        rest_n_h_text = n_h_texts[1:]

        non_hate_texts.append(source_n_h_text)
        non_hate_context.append("::".join(rest_n_h_text))

        hate_texts.append(h_texts[0])
        rest_h_text = n_h_texts + eval(j['response'])

        hate_context.append("::".join(rest_h_text))
    except:
        pass

In [343]:
gab_final = pd.DataFrame()
gab_final['text'] = hate_texts + non_hate_texts
gab_final['comments'] = hate_context + non_hate_context
gab_final['labels'] = ["HATE"]*len(hate_context) + ["NON_HATE"]*len(non_hate_context)

In [345]:
gab_final = gab_final.sample(frac= 1)

In [347]:
gab_final.to_csv("jing_gab_amr/gab.csv", index = False)

In [348]:
gab_final['labels'].value_counts()

NON_HATE    7665
HATE        7665
Name: labels, dtype: int64

# jing_twitter

NON_HATE    2490
HATE        2488

In [350]:
reddit = pd.read_csv("jing_reddit_amr/reddit.csv")

In [351]:
reddit = reddit.dropna()

In [352]:
hate_texts, hate_context, non_hate_texts, non_hate_context = [], [], [], []
for i, j in reddit.iterrows():
    j['text'] = j['text'].replace('\t', '')
    texts = j['text'].strip().split('\n')
    new_texts = []
    for p in texts:
        new_texts.append(pro.clean(".".join(p.split(".")[1:]).strip()))
    texts = new_texts
    idx = eval(j['hate_speech_idx']) 

    h_texts  = [v for i, v in enumerate(texts) if i+1 in idx]
    n_h_texts = [v for i, v in enumerate(texts) if i+1 not in idx]

    try:
        source_n_h_text = n_h_texts[0]
        rest_n_h_text = n_h_texts[1:]

        non_hate_texts.append(source_n_h_text)
        non_hate_context.append("::".join(rest_n_h_text))

        hate_texts.append(h_texts[0])
        rest_h_text = n_h_texts + eval(j['response'])

        hate_context.append("::".join(rest_h_text))
    except:
        pass

In [353]:
reddit_final = pd.DataFrame()
reddit_final['text'] = hate_texts + non_hate_texts
reddit_final['comments'] = hate_context + non_hate_context
reddit_final['labels'] = ["HATE"]*len(hate_context) + ["NON_HATE"]*len(non_hate_context)

In [354]:
reddit_final = reddit_final.sample(frac= 1)

In [356]:
reddit_final['labels'].value_counts()

NON_HATE    2490
HATE        2488
Name: labels, dtype: int64

In [358]:
reddit_final.to_csv("jing_reddit_amr/reddit.csv", index = False)

# twiiter16 and twitter15

In [457]:
import preprocessor as p

In [458]:
twitter = glob.glob("twitter15_amr/comments/*.json")
len_comments = []
df = pd.DataFrame()
id_list, comments_list = [], []
for lv, news in enumerate(twitter):
    all_comments= []
    user_comment = json.load(open(news, 'rb'))
    for i in user_comment['data']:
        all_comments.append(p.clean(i['text']))

    if len(all_comments)>0:#considering only news articles with atleast one comment
        processed = "::".join(all_comments)
        comments_list.append(processed)
        id_list.append(news[:-1][news[:-1].rfind("/")+1:].split(".")[0])
    print(lv, end = '\r', flush = True)

In [459]:
df['id'] =  id_list
df['comments'] = comments_list

In [460]:
label = pd.read_csv("twitter15_amr/label.txt", sep = ':')
text = pd.read_csv("twitter15_amr/source_tweets.txt", sep = '\t')

In [461]:
label.columns = ['label', 'id']
text.columns = ['id', 'text']

In [462]:
id2label = dict(list(zip(label['id'], label['label'])))
id2text = dict(list(zip(text['id'], text['text'])))

In [463]:
df['label'] = None
df['text'] = None
for i, j in df.iterrows():
    j['label'] = id2label.get(int(j['id']), None)
    j['text'] = p.clean(id2text.get(int(j['id']), None))

In [464]:
df = df.dropna()

In [465]:
df = df[['id', 'text', 'comments', 'label']]

In [466]:
df.to_csv("twitter15_amr/twitter15.csv", index = False)

In [468]:
df

,id,text,comments,label
0,427944719612915712,ex-marlboro man dies from smoking-related dise...,: Ex-Marlboro man dies from smoking-related di...,true
1,553264892799488000,"three more passport stamps, three more data ce...",e::Go to confirm that you re + enter email mak...,false
2,531108688749404160,news that 'home alone' star macauley culkin is...,Hahahahahahaha,false
3,524928878075457536,soldier shot at national war memorial in ottaw...,Just Hurd very heart breaking !! My heart goes...,true
4,531565935128498176,"hey, it looks like... oh. rt : photo of vladim...",Yeah ...eat and not freeze to death .... what ...,false
...,...,...,...,...
538,377519445578895360,ahh!! has an encyclopedia in the works for !! ...,FINALLY WOOOWOWOO::fhjdjdjkdkodopglg::fangirl:...,false
539,535315394127728641,raven symone blasts bill cosby rape story URL URL,I thought that when something like that happen...,false
540,489841070818488320,'nine britons and children' feared dead after ...,Of course. Ppl can detach fm that which is hap...,false
541,510918747188514816,breaking: video claims to show beheading of br...,Is that so they have free reign and can kill i...,true


# pheme

non_rumour    4023
rumour        2402
Name: labels, dtype: int64

In [39]:
import json
import glob
import pandas as pd
import preprocessor as pro

In [12]:
non_rumours = glob.glob("pheme_amr/all-rnr-annotated-threads/*/non-rumours/*/")
rumours = glob.glob("pheme_amr/all-rnr-annotated-threads/*/rumours/*/")

In [43]:
all_rumours = []
all_source_rumours = []
all_comments_rumours = []
for r in rumours:
    name = r[:-1][r[:-1].rfind("/")+1:]
    comments = glob.glob(r+"reactions/*.json")
    comment_list = []
    for c in comments:
        j_file = json.load(open(c, 'rb'))
        comment_list.append(pro.clean(j_file['text']))
    combined = "::".join(comment_list)
    all_comments_rumours.append(combined)
    all_rumours.append(r)
    all_source_rumours.append(pro.clean(json.load(open(r+f"source-tweets/{name}.json", 'rb'))['text']))
        

In [47]:
all_non_rumours = []
all_non_source_rumours = []
all_non_comments_rumours = []
for r in non_rumours:
    name = r[:-1][r[:-1].rfind("/")+1:]
    comments = glob.glob(r+"reactions/*.json")
    comment_list = []
    for c in comments:
        j_file = json.load(open(c, 'rb'))
        comment_list.append(pro.clean(j_file['text']))
    combined = "::".join(comment_list)
    all_non_comments_rumours.append(combined)
    all_non_rumours.append(r)
    all_non_source_rumours.append(pro.clean(json.load(open(r+f"source-tweets/{name}.json", 'rb'))['text']))
        

In [51]:
df = pd.DataFrame()
df['text'] = all_source_rumours + all_non_source_rumours
df['comments'] = all_comments_rumours + all_non_comments_rumours
df['labels'] = ['rumour']*len(all_rumours) + ['non_rumour']*len(all_non_rumours)

In [53]:
df = df.sample(frac=1)

In [55]:
df['labels'].value_counts()

non_rumour    4023
rumour        2402
Name: labels, dtype: int64

In [60]:
del df['id']

In [62]:
df['id'] = "pheme_"+df.index.astype(str)

In [64]:
df = df[['id', 'text', 'comments', 'labels']]

In [66]:
df.to_csv("pheme_amr/pheme.csv", index = False)

In [67]:
df

,id,text,comments,labels
4368,pheme_4368,"Just fyi, if you're using a hostage situation ...",It was a pew research poll on all news channel...,non_rumour
5645,pheme_5645,"New cops wear blue not camo, mingle and talk t...",really think getting control of policing out o...,non_rumour
544,pheme_544,"One pilot in crash was locked out of cockpit, ...",Cockpit! is that the same problem Air France h...,rumour
499,pheme_499,Killed Charlie Hebdo suspects came out firing ...,I dont no::Got killed::: Killed Charlie Hebdo ...,rumour
4852,pheme_4852,charges of sexual assault against Sydney hosta...,cops storming the place right now::sex assault...,non_rumour
...,...,...,...,...
5852,pheme_5852,"Also, the National Association of Black Psycho...",,non_rumour
1700,pheme_1700,The other day I asked a manager at the Market ...,thanks. Now we need a pic of what michael brow...,rumour
2597,pheme_2597,Love will win from hatred,". Bro! Twitter ""Verify"" ?!::"" ""::lol::"" "" "" ?""...",non_rumour
217,pheme_217,MORE: Police surround kosher grocery store in ...,*sigh* smh MORE: Police surround kosher grocer...,rumour


# rumoureval

In [107]:
import json
train = json.load(open("rumoureval_amr/train.json", 'rb'))
test = json.load(open("rumoureval_amr/test.json", 'rb'))
val = json.load(open("rumoureval_amr/val.json", 'rb'))

In [111]:
reval = train + test + val

In [117]:
all_comments = []
all_text = []
all_labels = []
for sample in reval:
    all_comments.append("::".join([pro.clean(i) for i in sample['comments']]))
    all_labels.append(sample['label'])
    all_text.append(pro.clean(sample['content']))

In [118]:
df = pd.DataFrame()
df['text'] = all_text
df['labels'] = all_labels
df['comments'] = all_comments 

In [121]:
df.to_csv("rumoureval_amr/rumoureval.csv", index=False)

# convert all datasets to same format

In [68]:
import pandas as pd

In [72]:
df = pd.read_csv("fake-emo_amr/fake-emo.csv")

In [75]:
df = df.rename(columns={'replies':'comments'})

In [94]:
for i, j in df.iterrows():
    df.at[i, 'comments'] = "::".join(eval(j['comments']))

In [95]:
df.to_csv("figlang_twitter_amr/figlang_twitter.csv", index = False)

In [16]:
import numpy as np

In [19]:
datasets = ['politifact', 'gossipcop']
for dataset in datasets:
    df = pd.read_csv(f"{dataset}_amr/{dataset}.tsv", sep = '\t')
    print(df.shape[0])

415
2813


In [146]:
datasets = ['antivax', 'fake-emo', 'jing_gab', 'jing_reddit', 'figlang_reddit', 'figlang_twitter','hasoc', 'pheme', 'twitter15', 'twitter16', 'rumoureval']
for dataset in datasets:
    df = pd.read_csv(f"{dataset}_amr/{dataset}.csv")
    print(dataset, df.shape[0], '\n', df['labels'].value_counts(), end = '\n\n*****************************\n')

antivax 3797 
 0    2865
1     932
Name: labels, dtype: int64

*****************************
fake-emo 7012 
 real    3793
fake    3219
Name: labels, dtype: int64

*****************************
jing_gab 15330 
 NON_HATE    7665
HATE        7665
Name: labels, dtype: int64

*****************************
jing_reddit 4978 
 NON_HATE    2490
HATE        2488
Name: labels, dtype: int64

*****************************
figlang_reddit 4400 
 NOT_SARCASM    2200
SARCASM        2200
Name: labels, dtype: int64

*****************************
figlang_twitter 4400 
 NOT_SARCASM    2200
SARCASM        2200
Name: labels, dtype: int64

*****************************
hasoc 712 
 NONE    414
HOF     298
Name: labels, dtype: int64

*****************************
pheme 6425 
 non_rumour    4023
rumour        2402
Name: labels, dtype: int64

*****************************
twitter15 543 
 True     276
False    267
Name: labels, dtype: int64

*****************************
twitter16 362 
 False    199
True     163
N

antivax 3797 
 0    2865
1     932
Name: labels, dtype: int64

*****************************
fake-emo 7012 
 real    3793
fake    3219
Name: labels, dtype: int64

*****************************
jing_gab 15330 
 NON_HATE    7665
HATE        7665
Name: labels, dtype: int64

*****************************
jing_reddit 4978 
 NON_HATE    2490
HATE        2488
Name: labels, dtype: int64

*****************************
figlang_reddit 4400 
 NOT_SARCASM    2200
SARCASM        2200
Name: labels, dtype: int64

*****************************
figlang_twitter 5000
SARCASM        2500
NOT_SARCASM    2500
Name: labels, dtype: int64

*****************************
hasoc 712 
 NONE    414
HOF     298
Name: labels, dtype: int64

*****************************
pheme 6425 
 non_rumour    4023
rumour        2402
Name: labels, dtype: int64

*****************************
twitter15 543 
 True     276
False    267
Name: labels, dtype: int64

*****************************
twitter16 362 
 False    199
True     163
Name: labels, dtype: int64

*****************************
rumoureval 446 
 real          185
fake          138
unverified    123
Name: labels, dtype: int64

*****************************

# finding max comments and sentences

In [3]:
import pandas as pd
import numpy as np

In [5]:
cd btp/amr

/home/karish19471/btp/amr


# comments

In [9]:
for data in ['gossipcop']:
    df = pd.read_csv(f"{data}_amr/{data}.tsv", sep =  '\t')
    lens = df['num_comments']
    print(f"{data}:- \t mean: {np.mean(lens)}, max: {np.max(lens)}, min: {np.min(lens)}, std: {np.std(lens)}")

gossipcop:- 	 mean: 19.67223604692499, max: 50, min: 3, std: 18.228823638570177


In [11]:
ls

amr2dgl.ipynb            fake-emo_amr/         jing_reddit_amr/
amr2hgnn_format.ipynb    figlang_reddit_amr/   pca_embedding_8.txt
amr_coref/               figlang_twitter_amr/  pheme_amr/
amr_generation.ipynb     gen_amr.py            politifact_amr/
amr_preprocessing.ipynb  gossipcop_amr/        rumoureval_amr/
antivax_amr/             hasoc_amr/            twitter15_amr/
data_processing.ipynb    jing_gab_amr/         twitter16_amr/


In [15]:
import glob
file = glob.glob('politifact_amr/politifact_amr_raw/politifact*.csv')
lens = []
for data in file:
    df = pd.read_csv(data)
    lens.append(df.shape[0])
lens = np.array(lens)
print(f"{data}:- \t mean: {np.mean(lens)}, max: {np.max(lens)}, min: {np.min(lens)}, std: {np.std(lens)}")

politifact_amr/politifact_amr_raw/politifact15342.csv:- 	 mean: 28.47342995169082, max: 50, min: 1, std: 20.560221885578564


In [10]:
for data in ['antivax', 'figlang_twitter', 'figlang_reddit', 'twitter16', 'rumoureval', 'pheme', 'twitter15', 'hasoc']:
    df = pd.read_csv(f"{data}_amr/{data}.csv")
    comments = df['comments']
    lens = []
    for i, j in df.iterrows():
        try:
            lens.append(len(j['comments'].split("::")))
        except:
            pass
    lens = np.array(lens)
    print(f"{data}:- \t mean: {np.mean(lens)}, max: {np.max(lens)}, min: {np.min(lens)}, std: {np.std(lens)}")

antivax:- 	 mean: 8.046204620462046, max: 493, min: 1, std: 31.002121354826425
figlang_twitter:- 	 mean: 3.8676, max: 20, min: 2, std: 3.218457742459888
figlang_reddit:- 	 mean: 2.4911363636363637, max: 8, min: 2, std: 0.756134655844179
twitter16:- 	 mean: 27.56145251396648, max: 486, min: 1, std: 48.697049686500854
rumoureval:- 	 mean: 17.546067415730338, max: 195, min: 1, std: 22.962221323954843
pheme:- 	 mean: 17.29544264012572, max: 345, min: 1, std: 19.983152110337084
twitter15:- 	 mean: 8.567107750472589, max: 500, min: 1, std: 24.126316439585658
hasoc:- 	 mean: 9.777272727272727, max: 272, min: 1, std: 31.973100888727878


# sentences

In [11]:
from nltk.tokenize import sent_tokenize

In [13]:
for data in ['antivax', 'figlang_twitter', 'figlang_reddit', 'twitter16', 'rumoureval', 'pheme', 'twitter15', 'hasoc']:
    df = pd.read_csv(f"{data}_amr/{data}.csv")
    lens = []
    for i, j in df.iterrows():
        try:
            lens.append(len(sent_tokenize(j['text'])))
        except:
            pass
    lens = np.array(lens)
    print(f"{data}:- \t mean: {np.mean(lens)}, max: {np.max(lens)}, min: {np.min(lens)}, std: {np.std(lens)}")

antivax:- 	 mean: 1.7368975506979194, max: 9, min: 1, std: 0.8688203132253286
figlang_twitter:- 	 mean: 2.0704, max: 10, min: 1, std: 1.2710011172300362
figlang_reddit:- 	 mean: 1.0079545454545455, max: 3, min: 1, std: 0.0913554383477113
twitter16:- 	 mean: 1.5165745856353592, max: 5, min: 1, std: 0.7403160301840561
rumoureval:- 	 mean: 1.4080717488789238, max: 8, min: 1, std: 0.7747861584105995
pheme:- 	 mean: 1.3771206225680934, max: 8, min: 1, std: 0.6576975203326774
twitter15:- 	 mean: 1.5498154981549817, max: 5, min: 1, std: 0.7968514859635492
hasoc:- 	 mean: 1.609375, max: 7, min: 1, std: 0.9733509692680231


# Annotation

In [20]:
import pandas as pd
import glob
import numpy as np

In [23]:
fake = glob.glob("annotation/fake/*.csv")
real = glob.glob("annotation/real/*.csv")

In [25]:
print("Total annotated files:- ", len(fake) + len(real))

Total annotated files:-  357


In [51]:
all_labels = set()
for i in fake + real:
    df = pd.read_csv(i)
    df['label'] = df['label'].astype(str).apply(lambda x: x.lower())
    all_labels = all_labels.union(set(df['label'].unique()))

In [52]:
all_labels

{'-',
 'false',
 'igmored',
 'ignored',
 'nan',
 'noies',
 'noise',
 'non_check_worthy',
 'non_check_wrthy',
 'non_checkworthy',
 'nosie',
 'quote',
 'true',
 'un_check_worthy',
 'uncheckworthy',
 'unverified'}

In [53]:
true = ['true']
false = ['false']
ignored = ['igmored', 'ignored', '-', 'nan']
noise = ['nosie', 'noies', 'noise']
quote = ['quote']
unverified = ['unverified']
non_check_worthy = ['un_check_worthy','uncheckworthy', 'non_check_worthy', 'non_check_wrthy', 'non_checkworthy']

In [55]:
import os

In [61]:
lv = 0
for i in real:
    df = pd.read_csv(i)
    df['label'] = df['label'].astype(str).apply(lambda x: x.lower())
    labels = []
    for sample in df['label']:
        if sample in true:
            labels.append('true')
        elif sample in false:
            labels.append('false')
        elif sample in ignored:
            labels.append('ignored')
        elif sample in quote:
            labels.append('quote')
        elif sample in unverified:
            labels.append('unverified')
        elif sample in non_check_worthy:
            labels.append('non_check_worthy')
        elif sample in noise:
            labels.append('noise')

    name = i[i.rfind("/")+1:]
    df['label'] = np.array(labels)
    dst = "annotation_processed/real/"
    os.makedirs(dst, exist_ok= True)
    df.to_csv(dst+name)
    print(lv, end = '\r', flush = True)
    lv+=1

In [75]:
import re
fake = glob.glob("annotation_processed/fake/*.csv")
real = glob.glob("annotation_processed/real/*.csv")
all_labels = set()
for i in fake + real:
    prefix = i[:i.rfind("/")+1]
    suffix = i[i.rfind("/")+1:]
    os.rename(i, prefix + suffix.replace("-", "_").split("_")[0].strip()+".csv")